In [4]:
import os
from datasets import load_dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer
import evaluate
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# dataset_name = 'beomi/KoAlpaca-v1.1a'
# dataset_name = 'nlpai-lab/kullm-v2'
dataset_name = 'heegyu/OIG-small-chip2-ko'
model_name = 'EleutherAI/polyglot-ko-1.3b'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token':'<|endoftext|>'})

dataset = load_dataset(dataset_name)
dataset = dataset.filter(lambda example: len(example['user_translated'])>0, num_proc=24) # filtering under 0 token length

# filtering over 1024 token length
def encode_preprocess(examples):
    return tokenizer(examples['user_translated'])#, padding=True, return_tensors='pt')

# truncate max length, add padding true
def encode_pad_preprocess(examples):
    return tokenizer(examples['user_translated'], max_length=1024, truncation=True, padding=True, return_tensors='pt')

# extract truncated sentence max_length 1024
def decode_process(examples):
    return {'trunc_instruction': tokenizer.decode(examples['input_ids'], skip_special_tokens=True)}

trunc_data = dataset.map(encode_preprocess, batched=True, num_proc=24)
trunc_data = trunc_data.filter(lambda example: len(example['input_ids'])<1024, num_proc=24)
encode_pad_data = trunc_data.map(encode_pad_preprocess, batched=True, num_proc=24)
preprocessed_data = encode_pad_data.map(decode_process, num_proc=96)
print(preprocessed_data)

Found cached dataset json (/home/uj-user/.cache/huggingface/datasets/heegyu___json/heegyu--OIG-small-chip2-ko-143a9b79176d5f9d/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/uj-user/.cache/huggingface/datasets/heegyu___json/heegyu--OIG-small-chip2-ko-143a9b79176d5f9d/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-5f9d0590ba022ec6_*_of_00024.arrow
Loading cached processed dataset at /home/uj-user/.cache/huggingface/datasets/heegyu___json/heegyu--OIG-small-chip2-ko-143a9b79176d5f9d/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-55462aad54e6674b_*_of_00024.arrow
Loading cached processed dataset at /home/uj-user/.cache/huggingface/datasets/heegyu___json/heegyu--OIG-small-chip2-ko-143a9b79176d5f9d/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-28f0f874308a377b_*_of_00024.arrow
Loading cached processed dataset at /home/uj-user/.cache/huggingface/datasets/heegyu___json/heegyu--OIG-small-chip2-ko-143a9b79176d5f9d/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-da0a5cc49e5b3ffc_*_of_00024.arrow
Loading cach

DatasetDict({
    train: Dataset({
        features: ['user', 'chip2', 'index', 'user_translated', 'chip2_translated', 'input_ids', 'token_type_ids', 'attention_mask', 'trunc_instruction'],
        num_rows: 210282
    })
})


In [5]:
preprocessed_data

DatasetDict({
    train: Dataset({
        features: ['user', 'chip2', 'index', 'user_translated', 'chip2_translated', 'input_ids', 'token_type_ids', 'attention_mask', 'trunc_instruction'],
        num_rows: 210282
    })
})

In [6]:
perplexity = evaluate.load("perplexity", module_type="metric")
instruction = preprocessed_data['train']['trunc_instruction']
instruction

len_instruction = [len(text) for text in instruction]
ppl_results = perplexity.compute(model_id=model_name, add_start_token=False, predictions=instruction)
round_ppl_results = [round(ppl, 2) for ppl in  ppl_results['perplexities']]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/13143 [00:00<?, ?it/s]

In [7]:
instruction

['유언장이 있는 것이 좋다는 말을 들었습니다. 유언장이란 무엇입니까?\n\n',
 '일자리는 어떻게 찾나요?\n\n',
 '돔형 천장, 기둥 및 문이 있는 맨션 내부의 우울한 장면에 대한 상세한 서면 설명을 생성합니다.\n\n',
 '저는 발목이 삔 상태이고 더 빨리 낫도록 돕기 위해 제가 무엇을 할 수 있는지 알아야 합니다.\n\n',
 '칠면조는 어떻게 구우나요?\n\n',
 '집에서 직접 데리야끼 소스를 만드는 방법은 무엇입니까?\n\n',
 '나는 구순구개열이 있다. 성형 수술을 받아야합니까?\n\n',
 '밀레니얼 세대에 대한 이야기를 많이 들었습니다. 그들에 대해 더 말씀해 주시겠습니까?\n\n',
 '최초의 여성 참정권 대회는 몇 년도에 열렸습니까?\n\n',
 '볼만한 새 영화를 찾고 있어요. 좋은 것을 제안할 수 있습니까?\n\n',
 '미국은 왜 금지령을 내렸습니까?\n\n',
 '관장과 인공 항문의 차이점에 대해 약간 혼란 스럽습니다. 어느 것이 나에게 적합합니까?\n\n',
 '나는 항상 우주 비행사가 되고 싶었지만 내가 충분히 잘할 것이라고는 생각하지 못했습니다.\n\n',
 '은퇴하려면 얼마나 많은 돈을 저축해야 합니까?\n\n',
 '소기업을 위한 가장 중요한 디지털 마케팅 전략은 무엇입니까?\n\n',
 '어떤 종류의 세금 공제를 받을 수 있습니까?\n\n',
 '정화조 시스템의 가장 일반적인 유형은 무엇입니까?\n\n',
 '상인 허가증, 양탄자, 나무 테이블이 있는 상점 내부의 음산한 장면에 대한 간결한 위치 설명을 작성합니다.\n\n',
 '한 공장에서 야간 근무를 위해 경비원 그룹을 고용했습니다. 4명의 경비원은 9시간의 야간 근무를 위해 순환 일정에 동의했습니다. 첫 번째 경비원은 아직 깨어 있기 때문에 3시간이 걸리고, 마지막 경비원은 일찍 일어나 2시간이 걸리며, 중간 두 경비원은 남은 시간을 나누어서 잡습니다. 각 미들 가드는 몇 시간이 걸립니까?\n\n',
 '시체가 있는 섬뜩한 숲 장면에 대한 간략한 설명을 생성

In [8]:
dict_ppl_instruction = dict(zip(instruction, round_ppl_results))
df_dataset = pd.DataFrame(sorted(dict_ppl_instruction.items(), key=lambda x: x[0]), columns=['instruction', 'ppl'])
df_dataset

,instruction,ppl
0,"""30 Rock""에서 연기한 Denis O'Hare의 이름은 무엇입니까?\n\n",161.76
1,"""90년대 댄스 뮤직의 리틀 리차드""라고 불리는 사람의 이름은 무엇입니까?\n\n",140.36
2,"""<pad> <ac\n\n",88731.02
3,"""A Chorus Line""의 투어링 회사 이름은 무엇입니까?\n\n",522.38
4,"""Afrikaans medium decree""를 발표한 Bantu 교육 개발부 장관...",151.52
...,...,...
130022,힙에서 값을 삭제하는 Python 함수를 제공하십시오.\n\n,503.96
130023,힙에서 값을 삭제합니다.\n\n,6989.41
130024,힙의 요소에 액세스하기 위한 Python 함수를 작성하십시오.\n\n,146.61
130025,힙의 요소에 액세스합니다.\n\n,3417.16


In [9]:
datasetname = dataset_name.split('/')[-1]
df_dataset.to_json(f"{datasetname}_ppl.json", orient='records', lines=True, force_ascii=False)

In [10]:
from datasets import Dataset, DatasetDict

dataset = preprocessed_data['train'].add_column("ppl", round_ppl_results)
dataset = dataset.add_column("len", len_instruction)
dataset = dataset.remove_columns(['input_ids', 'attention_mask', 'trunc_instruction',])
dataset

Dataset({
    features: ['user', 'chip2', 'index', 'user_translated', 'chip2_translated', 'token_type_ids', 'ppl', 'len'],
    num_rows: 210282
})

In [11]:
datasetname = dataset_name.split('/')[-1]
model_name = model_name.split('/')[-1]
dataset.push_to_hub(f'nayohan/{datasetname}_ppl_{model_name}')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/211 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]